# Generating preliminary results for the final aggregation classifier

In [8]:
import os

# Function to change to the parent directory
def change_to_parent_directory():
    # Check if the directory has already been changed
    if not os.environ.get('DIR_CHANGED'):
        try:
            current_dir = os.path.dirname(os.path.abspath(__file__))
        except NameError:
            current_dir = os.getcwd()
        parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
        os.chdir(parent_dir)
        os.environ['DIR_CHANGED'] = '1'
        print(f"Current working directory changed to: {os.getcwd()}")
    else:
        print("Directory has already been changed.")

# Call the function to change the working directory
change_to_parent_directory()

Directory has already been changed.


## Optional: Create testing dataset
Note: If you want do to this, set create_test_parquet to **True**

In [9]:
create_test_parquet = True

if create_test_parquet:
    import pandas as pd
    import numpy as np

    # List of input Parquet files along with their maximum rows and desired labels
    input_files = [
        {'file': 'testdata/phishing_4.parquet', 'label': 'phishing', 'max_rows': 50000},
        {'file': 'testdata/benign_2312.parquet', 'label': 'benign', 'max_rows': 150000},
        {'file': 'testdata/umbrella_benign_FINISHED.parquet', 'label': 'benign', 'max_rows': 150000},
        {'file': 'testdata/malware_3.parquet', 'label': 'malware', 'max_rows': 50000},
        {'file': 'testdata/lex-dga-830k-pick.parquet', 'label': 'dga', 'max_rows': 50000},
    ]

    # Read the first file to get the initial columns and create the first dataframe
    first_file_info = input_files[0]
    combined_df = pd.read_parquet(first_file_info['file']).head(first_file_info['max_rows'])
    
    # Overwrite the "label" column with the specified label for the first file
    combined_df['label'] = first_file_info['label']
    
    # Get the columns from the first dataframe
    all_columns = combined_df.columns.tolist()

    # Process the remaining files
    for file_info in input_files[1:]:
        df = pd.read_parquet(file_info['file']).head(file_info['max_rows'])
        
        # Overwrite the "label" column with the specified label
        df['label'] = file_info['label']
        
        # Ensure all columns from the first dataframe are present
        for col in all_columns:
            if col not in df.columns:
                df[col] = -1
        
        # Align the dataframe to the columns of the first dataframe
        df = df[all_columns]
        
        # Append the dataframe to the combined dataframe
        combined_df = pd.concat([combined_df, df], ignore_index=True)


    ROW_LIMIT = 100000 # (0 - no limit)

    # Subsample the combined dataframe to take N domains
    combined_df = combined_df.sample(n=ROW_LIMIT, random_state=42)
    
    # Shuffling the DataFrame
    shuffled_df = combined_df.sample(frac=1)

    # Save the selected rows to a new Parquet file
    shuffled_df.to_parquet('testdata/decision_training_xx.parquet')


/tmp/ipykernel_277894/1339135024.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = -1
/tmp/ipykernel_277894/1339135024.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = -1
/tmp/ipykernel_277894/1339135024.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = -1
/tmp

In [10]:
shuffled_df

,domain_name,label,dns_has_dnskey,dns_A_count,dns_AAAA_count,dns_MX_count,dns_NS_count,dns_TXT_count,dns_SOA_count,dns_CNAME_count,...,rdap_ip_v4_count,rdap_ip_v6_count,rdap_ip_shortest_v4_prefix_len,rdap_ip_longest_v4_prefix_len,rdap_ip_shortest_v6_prefix_len,rdap_ip_longest_v6_prefix_len,rdap_ip_avg_admin_name_len,rdap_ip_avg_admin_name_entropy,rdap_ip_avg_admin_email_len,rdap_ip_avg_admin_email_entropy
54451,57w0-alternate.app.link,benign,0.0,4,8,0,0,0,0,0,...,4,7,17,17,28,28,11.916667,0.228384,21.083333,0.137391
247717,5lovelanguages.com,benign,0.0,2,2,1,2,4,1,0,...,10,8,12,18,32,32,9.333333,0.340212,16.888889,0.169547
15621,brannanlaw9.wixsite.com,phishing,0.0,0,0,0,0,0,0,1,...,1,0,10,10,0,0,10.000000,0.292193,23.000000,0.157442
255533,e673.dsce9.akamaiedge.net,benign,0.0,1,5,0,0,0,0,0,...,1,5,20,20,48,48,0.000000,0.000000,0.000000,0.000000
89645,camwhores.tube,benign,0.0,1,0,0,2,0,1,0,...,3,2,23,24,32,32,18.800000,0.187118,34.400000,0.072169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343814,revolutionbeauty.us,benign,0.0,1,0,1,4,5,1,0,...,7,4,13,24,32,48,18.363636,0.155818,36.000000,0.090876
104123,wcedonline.westerncape.gov.za,benign,0.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
380872,idg88.xyz,malware,0.0,1,0,3,2,1,1,0,...,1,0,21,21,0,0,13.000000,0.272815,20.000000,0.179209
275135,zingat.com,benign,0.0,1,0,1,2,3,1,0,...,9,6,13,27,32,32,9.866667,0.284419,15.466667,0.150532


In [11]:
df.dtypes

domain_name                        object
label                              object
dns_has_dnskey                      int64
dns_A_count                         int64
dns_AAAA_count                      int64
                                    ...  
rdap_ip_longest_v6_prefix_len       int64
rdap_ip_avg_admin_name_len          int64
rdap_ip_avg_admin_name_entropy      int64
rdap_ip_avg_admin_email_len         int64
rdap_ip_avg_admin_email_entropy     int64
Length: 178, dtype: object

## Optional: Generate preliminary results for training the final aggregation classifier

In [12]:
import pandas as pd
import numpy as np
from classifiers.pipeline import Pipeline

# Initialize the classification pipeline
clf = Pipeline()

# Read the input parquet file
input_df = pd.read_parquet('testdata/decision_training_xx.parquet')

preliminary_results_df = clf.generate_preliminary_results(input_df, add_final=False, output_file="preliminary_results_x.parquet")

In [13]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = '{:.6f}'.format
preliminary_results_df

,domain_name,dns_available,dns_nonzero,tls_available,tls_nonzero,ip_available,ip_nonzero,rdap_available,rdap_nonzero,geo_available,geo_nonzero,label,phishing_cnn_result,phishing_lgbm_result,phishing_xgboost_result,phishing_deepnn_result,phishing_dns_nn_result,phishing_rdap_nn_result,malware_cnn_result,malware_lgbm_result,malware_xgboost_result,dga_binary_nn_result,phishing_sum,phishing_avg,phishing_prod,malware_sum,malware_avg,malware_prod,total_sum,total_avg,total_prod
54451,57w0-alternate.app.link,0.600000,0.225000,1.000000,0.833333,1.000000,0.750000,1.000000,0.833333,1.000000,0.777778,benign,1.000000,0.008122,0.002815,0.001367,0.066331,0.013630,1.000000,0.000191,0.002436,0.000000,1.092264,0.182044,0.000000,1.002627,0.334209,0.000000,2.094891,0.209489,0.000000
247717,5lovelanguages.com,1.000000,0.750000,0.833333,0.291667,1.000000,1.000000,1.000000,0.708333,1.000000,1.000000,benign,0.000000,0.000682,0.000621,0.000358,0.007201,0.002786,0.000000,0.000052,0.002233,0.003491,0.011647,0.001941,0.000000,0.002285,0.000762,0.000000,0.017423,0.001742,0.000000
15621,brannanlaw9.wixsite.com,0.600000,0.225000,1.000000,0.833333,1.000000,0.375000,1.000000,0.708333,1.000000,0.722222,phishing,1.000000,0.794902,0.967936,0.999944,0.689681,0.941273,0.000000,0.000533,0.002408,0.000001,5.393736,0.898956,0.499459,0.002941,0.000980,0.000000,5.396678,0.539668,0.000000
255533,e673.dsce9.akamaiedge.net,0.900000,0.525000,0.833333,0.291667,1.000000,0.875000,0.833333,0.250000,1.000000,0.722222,benign,0.000000,0.000314,0.000381,0.000000,0.055679,0.000929,0.000000,0.000280,0.002364,0.000000,0.057303,0.009551,0.000000,0.002644,0.000881,0.000000,0.059947,0.005995,0.000000
89645,camwhores.tube,0.900000,0.550000,0.833333,0.291667,1.000000,1.000000,1.000000,0.708333,1.000000,1.000000,benign,0.000000,0.000668,0.000933,0.000633,0.071273,0.033787,0.000000,0.000061,0.002233,0.000502,0.107293,0.017882,0.000000,0.002294,0.000765,0.000000,0.110089,0.011009,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343814,revolutionbeauty.us,1.000000,0.700000,0.833333,0.291667,1.000000,1.000000,0.833333,0.416667,1.000000,1.000000,benign,1.000000,0.000353,0.000631,0.000125,0.003323,0.000713,1.000000,0.000052,0.002233,0.038546,1.005146,0.167524,0.000000,1.002286,0.334095,0.000000,2.045977,0.204598,0.000000
104123,wcedonline.westerncape.gov.za,0.600000,0.200000,1.000000,0.833333,1.000000,0.500000,0.833333,0.041667,1.000000,0.666667,benign,0.000000,0.000520,0.000777,0.000249,0.144647,0.010928,1.000000,0.000305,0.002364,0.000000,0.157121,0.026187,0.000000,1.002669,0.334223,0.000001,1.159789,0.115979,0.000000
380872,idg88.xyz,1.000000,0.725000,0.833333,0.291667,1.000000,0.625000,1.000000,0.708333,1.000000,0.722222,malware,0.000000,0.006750,0.010146,0.010546,0.051278,0.027170,0.000000,0.677820,0.568785,0.025597,0.105889,0.017648,0.000000,1.246605,0.415535,0.000000,1.378092,0.137809,0.000000
275135,zingat.com,1.000000,0.700000,0.833333,0.291667,1.000000,1.000000,1.000000,0.708333,1.000000,1.000000,benign,0.000000,0.000425,0.000540,0.002750,0.008847,0.002785,0.000000,0.000050,0.002233,0.727587,0.015347,0.002558,0.000000,0.002283,0.000761,0.000000,0.745217,0.074522,0.000000


In [14]:
preliminary_results_df[["domain_name", "label", "phishing_cnn_result", "phishing_lgbm_result", "malware_cnn_result", "malware_xgboost_result", "dga_binary_nn_result"]]

,domain_name,label,phishing_cnn_result,phishing_lgbm_result,malware_cnn_result,malware_xgboost_result,dga_binary_nn_result
54451,57w0-alternate.app.link,benign,1.000000,0.008122,1.000000,0.002436,0.000000
247717,5lovelanguages.com,benign,0.000000,0.000682,0.000000,0.002233,0.003491
15621,brannanlaw9.wixsite.com,phishing,1.000000,0.794902,0.000000,0.002408,0.000001
255533,e673.dsce9.akamaiedge.net,benign,0.000000,0.000314,0.000000,0.002364,0.000000
89645,camwhores.tube,benign,0.000000,0.000668,0.000000,0.002233,0.000502
...,...,...,...,...,...,...,...
343814,revolutionbeauty.us,benign,1.000000,0.000353,1.000000,0.002233,0.038546
104123,wcedonline.westerncape.gov.za,benign,0.000000,0.000520,1.000000,0.002364,0.000000
380872,idg88.xyz,malware,0.000000,0.006750,0.000000,0.568785,0.025597
275135,zingat.com,benign,0.000000,0.000425,0.000000,0.002233,0.727587
